In [1]:
import chemistry_psql as cp
import importlib
importlib.reload(cp)
import psycopg2
import pandas as pd

#dfm = pd.read_csv("../RBP_Manual.csv")
#dfm = pd.read_csv("../data-1633411346295.csv")
#print(type(dfm))
dfm = pd.read_csv("../Splicng_agents.csv")
print(dfm)

struct_dict = ['Smiles','canonical_smiles','smiles']
#print(dfm.columns)
#print()
#struct_col = set(struct_dict).intersection(dfm.columns)
struct_col = list(set(struct_dict).intersection(dfm.columns))[0]
print('Structure Column : ',struct_col)

struct = dfm[[struct_col]].drop_duplicates()
#print(struct)
#struct = struc.drop_duplicates()
#print(struct)

struct_list = struct.to_records(index = False)
#print(struct_list)

imp_struct = ()

print()
print('# # # Structure Insert # # #')
print()
for i in struct_list:
    #print(i)
    cp.SmilesInsert(i)


print()
print('# # # Structure Search # # #')
print()
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
for i in struct_list:
    j= i[0]
    print()
    print(j)
    sql = 'select * from public.structures where molecule =\' '+j+' \'::mol;'
    #print(sql)
    cur.execute(sql)
    mollist = cur.fetchall()
    print(mollist)



#test = ("C1CCCCOCCCC1","C1CCCCNCCCC1","C1CCCCSCCCC1")

#cp.SmilesInsert(test)

ModuleNotFoundError: No module named 'chemistry_psql'

In [3]:
from rdkit import Chem
from rdkit.Chem import SaltRemover as rdsr

remover = rdsr.SaltRemover()

# We can have a look at some of the salts
a = Chem.MolToSmarts(remover.salts[3])
print(a)
print(len(remover.salts))

for i in remover.salts:
    a = Chem.MolToSmarts(i)
    print(a)

N(=O)(-,:O)O
15
[Cl,Br,I]
[#3,#11,#19,#20,#12]
[O,N]
N(=O)(-,:O)O
P(=O)(-,:O)(-,:O)O
P(-,:F)(-,:F)(-,:F)(-,:F)(-,:F)F
S(=O)(=O)(-,:O)O
[C&H3]S(=O)(=O)-,:O
c1cc(-,:[C&H3])ccc1S(=O)(=O)-,:O
[C&H3]C(=O)O
FC(-,:F)(-,:F)C(=O)O
OC(=O)C=CC(=O)O
OC(=O)C(=O)O
OC(=O)C(-,:O)C(-,:O)C(=O)O
C1CCCCC1[N&H1]C1CCCCC1


In [5]:
from rdkit.Chem.MolStandardize import rdMolStandardize
un = rdMolStandardize.Uncharger()

for i in struct_list:
    j = Chem.MolFromSmiles(i[0])
    r = remover.StripMol(j)
    if j.GetNumAtoms() != r.GetNumAtoms():
        r = un.uncharge(r)
        k = Chem.MolToSmiles(r)
        print(i[0], ' - ',k)